In [1]:
#set up credentials + imports
from office365.runtime.auth.user_credential import UserCredential
from office365.sharepoint.client_context import ClientContext
from office365.sharepoint.client_context import ClientContext
from office365.sharepoint.files.system_object_type import FileSystemObjectType
import os
from dotenv import load_dotenv

#set up credentials
load_dotenv(".env", override=True)
userCredential = UserCredential(os.getenv('user'),os.getenv('password') )

In [3]:
from office365.sharepoint.taxonomy.field import TaxonomyField

def list_taxonomy_fields(itemlist):
    fields = itemlist.fields.get().execute_query()
    #get all fields
    taxonomy_fields = []
    for f in fields:  # type: Field
        #find only taxonomy types
        if type(f) is TaxonomyField:
            taxonomy_fields.append(f)
    return taxonomy_fields

In [6]:

def getalllists(ctx):
    lists = ctx.web.lists.get_all().execute_query()
    #print(lists)
    return lists

In [98]:
import json
def query_items_byfield(target_list, tax_field, baseurl):
    """
    :type target_list: office365.sharepoint.lists.list.List
    """
    
    #https://hksinc.sharepoint.com/sites/research/Shared%20Documents/ + LinkFilename is url
    items = target_list.items.get().select(["LinkFilename", tax_field.internal_name]).execute_query() #tax_field.internal_name, 
    for t in items:
            
        if {} not in t.properties.values():
            #json_object = json.dumps(t.properties) 
            t.properties['baseurl'] = baseurl
            with open("taxo/sample.json", "a") as outfile:
                json.dump(t.properties, outfile, ensure_ascii=False)
                outfile.write('\n')

In [99]:
import pandas as pd
#read from csv, clean and filter
sitesdf= pd.read_csv("knowledgesites.csv",keep_default_na=False)

#iterate sites        
for index, row in sitesdf.iterrows():
    site_url = row["url"]
    print(site_url)
    ctx = ClientContext(site_url).with_credentials(userCredential)
    try :
        lists = getalllists(ctx)
    except :
        continue
    for l in lists:
        itemlist = ctx.web.lists.get_by_title(l.title) #Research Contribution Library

        tax_fields = list_taxonomy_fields(itemlist)
        if(len(tax_fields)>0):
            folder = itemlist.root_folder.expand(["ServerRelativeURL"]).get().execute_query()
            baseurl = "https://hksinc.sharepoint.com/" + folder.serverRelativeUrl
            for t in tax_fields:
                
                query_items_byfield(itemlist, t, baseurl) 
           
           

https://hksinc.sharepoint.com/sites/research
https://hksinc.sharepoint.com/sites/Energy
https://hksinc.sharepoint.com/sites/EnergyModeling
https://hksinc.sharepoint.com/sites/EquitableCommunities
https://hksinc.sharepoint.com/sites/EquityinEducation
https://hksinc.sharepoint.com/sites/Flexibility
https://hksinc.sharepoint.com/sites/Gender
https://hksinc.sharepoint.com/sites/GIS
https://hksinc.sharepoint.com/sites/HealthandArt
https://hksinc.sharepoint.com/sites/HealthcareWorkers
https://hksinc.sharepoint.com/sites/Healthy
https://hksinc.sharepoint.com/sites/HealthyEnvironments
https://hksinc.sharepoint.com/sites/hpd
https://hksinc.sharepoint.com/sites/InclusiveDesign
https://hksinc.sharepoint.com/sites/JEDI
https://hksinc.sharepoint.com/sites/LeadingWithKnowledge
https://hksinc.sharepoint.com/sites/MassTimber
https://hksinc.sharepoint.com/sites/Materials/
https://hksinc.sharepoint.com/sites/MBH
https://hksinc.sharepoint.com/sites/natureofplace
https://hksinc.sharepoint.com/sites/Occupa